In [2]:
import cv2 as cv
import numpy as np
import glob
import matplotlib.pyplot as plt


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Users/hyunowk/opt/anaconda3/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/hyunowk/opt/anaconda3/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/Users/hyunowk/opt/anaconda3/lib/python3.9/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/hyunowk/opt/anaconda3/lib/python3.9/site-packages/traitlets/config/application.py", line 846, in launch_instance
    ap

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

In [ ]:
# 보라색 원이 포함된 augmentation된 이미지들의 경로 설정
image_folder = 'output_folder/*.jpg'
image_paths = glob.glob(image_folder)

In [ ]:
# 보라색 범위를 자동으로 탐지하기 위한 함수
def auto_detect_purple(image):
    hsv_image = cv.cvtColor(image, cv.COLOR_BGR2HSV)

    # 보라색 범위: 넓은 범위로 설정 (이후 최적화)
    lower_bound = np.array([120, 20, 20])
    upper_bound = np.array([160, 255, 255])

    # 초기 마스크 생성
    mask = cv.inRange(hsv_image, lower_bound, upper_bound)

    # 픽셀 값 추출
    hue_channel = hsv_image[:, :, 0]
    purple_hues = hue_channel[mask > 0]

    return purple_hues, mask

# 각 이미지에서 최적의 보라색 범위를 추출하고 시각화
def process_images(image_paths):
    all_purple_hues = []

    for idx, image_path in enumerate(image_paths):
        img = cv.imread(image_path)

        # 보라색 픽셀 탐지
        purple_hues, mask = auto_detect_purple(img)
        all_purple_hues.append(purple_hues)

        # 마스크 적용된 이미지 시각화
        purple_pixels = cv.bitwise_and(img, img, mask=mask)
        cv.imshow(f'Image {idx}', purple_pixels)

        # 마스크 저장
        cv.imwrite(f'output_purple_{idx}.jpg', purple_pixels)

        # 히스토그램 시각화 (보라색 hue 분포)
        plt.figure(figsize=(6, 4))
        plt.hist(purple_hues, bins=40, color='purple', alpha=0.7, label='Hue Distribution')
        plt.title(f'Image {idx} - Purple Hue Distribution')
        plt.xlabel('Hue Value')
        plt.ylabel('Frequency')
        plt.legend()
        plt.savefig(f'purple_histogram_{idx}.png')
        plt.show()

    return all_purple_hues

# 이미지 처리 및 보라색 히스토그램 생성
all_purple_hues = process_images(image_paths)



In [ ]:
import seaborn as sns

def process_images_kde(image_paths):
    all_purple_hues = []

    for idx, image_path in enumerate(image_paths):
        img = cv.imread(image_path)

        # 보라색 픽셀 탐지
        purple_hues, mask = auto_detect_purple(img)
        all_purple_hues.append(purple_hues)

        # 마스크 적용된 이미지 시각화
        purple_pixels = cv.bitwise_and(img, img, mask=mask)
        cv.imshow(f'Image {idx}', purple_pixels)

        # KDE를 사용해 부드러운 분포 그래프 생성
        plt.figure(figsize=(6, 4))
        sns.kdeplot(purple_hues, color='purple', fill=True, label='Purple Hue Distribution')
        plt.title(f'Image {idx} - Purple Hue KDE')
        plt.xlabel('Hue Value')
        plt.ylabel('Density')
        plt.legend()
        plt.savefig(f'purple_kde_{idx}.png')
        plt.show()

    return all_purple_hues

# 이미지 처리 및 보라색 hue KDE 생성
all_purple_hues_kde = process_images_kde(image_paths)
